In [36]:
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')

from PIL import Image
from tensorflow.keras.preprocessing import image, image_dataset_from_directory
from tensorflow.keras import models, layers, optimizers
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import BatchNormalization

import os
import gc
from timeit import default_timer as timer

In [2]:
train_dir = '/Users/arencarpenter/Desktop/Detecting_ALL_with_CNN/Data/training/'
test_dir = '/Users/arencarpenter/Desktop/Detecting_ALL_with_CNN/Data/testing/'
val_dir = '/Users/arencarpenter/Desktop/Detecting_ALL_with_CNN/Data/validation/'

In [25]:
train_g = image.ImageDataGenerator(rescale = 1/255).flow_from_directory(train_dir,
                                                                  target_size = (256,256), 
                                                                  color_mode='rgb',
                                                                  class_mode='binary')
val_g = image.ImageDataGenerator(rescale = 1/255).flow_from_directory(val_dir,
                                                                target_size = (256,256), 
                                                                color_mode='rgb',
                                                                class_mode='binary')
test_g = image.ImageDataGenerator(rescale = 1/255).flow_from_directory(test_dir,
                                                                target_size = (256,256), 
                                                                color_mode='rgb',
                                                                class_mode='binary')

Found 1346 images belonging to 2 classes.
Found 593 images belonging to 2 classes.
Found 598 images belonging to 2 classes.


In [26]:
model = models.Sequential()
model.add(layers.Conv2D(64, 3, activation = 'relu', input_shape=(256,256,3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation = 'relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [27]:
# compiling models
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['accuracy', 'Recall'])

In [28]:
# training
start = timer()

history = model.fit(train_g, 
                    epochs=5, 
                    validation_data=val_g, 
                    workers = 7
                   )

end = timer()
elapsed = end - start
print('Total Time Elapsed: ', int(elapsed//60), ' minutes ', (round(elapsed%60)), ' seconds')

Epoch 1/5
43/43 [==============================] - 327s 8s/step - loss: 0.7881 - accuracy: 0.6969 - recall: 0.7013 - val_loss: 0.5494 - val_accuracy: 0.7723 - val_recall: 0.7177
Epoch 2/5
43/43 [==============================] - 316s 7s/step - loss: 0.5513 - accuracy: 0.7385 - recall: 0.7177 - val_loss: 0.5161 - val_accuracy: 0.7605 - val_recall: 0.8061
Epoch 3/5
43/43 [==============================] - 307s 7s/step - loss: 0.4860 - accuracy: 0.7771 - recall: 0.7519 - val_loss: 0.5694 - val_accuracy: 0.7066 - val_recall: 0.8197
Epoch 4/5
43/43 [==============================] - 337s 8s/step - loss: 0.4168 - accuracy: 0.8061 - recall: 0.7741 - val_loss: 0.6853 - val_accuracy: 0.6341 - val_recall: 0.9524
Epoch 5/5
43/43 [==============================] - 347s 8s/step - loss: 0.3586 - accuracy: 0.8380 - recall: 0.8276 - val_loss: 0.5296 - val_accuracy: 0.7707 - val_recall: 0.7857
Total Time Elapsed:  28  minutes  2  seconds


In [29]:
model = models.Sequential()
model.add(layers.Conv2D(64, 3, activation = 'relu', input_shape=(256,256,3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(256, activation = 'relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['accuracy', 'Recall'])

start = timer()

history = model.fit(train_g, 
                    epochs=5, 
                    validation_data=val_g, 
                    workers = 7
                   )

end = timer()
elapsed = end - start
print('Total Time Elapsed: ', int(elapsed//60), ' minutes ', (round(elapsed%60)), ' seconds')

Epoch 1/5
43/43 [==============================] - 80s 2s/step - loss: 0.5749 - accuracy: 0.7303 - recall: 0.7266 - val_loss: 0.4986 - val_accuracy: 0.7690 - val_recall: 0.7415
Epoch 2/5
43/43 [==============================] - 83s 2s/step - loss: 0.5081 - accuracy: 0.7623 - recall: 0.7370 - val_loss: 0.5021 - val_accuracy: 0.7622 - val_recall: 0.7925
Epoch 3/5
43/43 [==============================] - 87s 2s/step - loss: 0.4794 - accuracy: 0.7868 - recall: 0.7608 - val_loss: 0.4925 - val_accuracy: 0.7808 - val_recall: 0.7823
Epoch 4/5
43/43 [==============================] - 85s 2s/step - loss: 0.4493 - accuracy: 0.7912 - recall: 0.7727 - val_loss: 0.5225 - val_accuracy: 0.7555 - val_recall: 0.8129
Epoch 5/5
43/43 [==============================] - 85s 2s/step - loss: 0.4235 - accuracy: 0.8076 - recall: 0.7890 - val_loss: 0.4975 - val_accuracy: 0.7656 - val_recall: 0.7415
Total Time Elapsed:  7  minutes  11  seconds


In [30]:
model = models.Sequential()
model.add(layers.Conv2D(64, 3, activation = 'relu', input_shape=(256,256,3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(256, activation = 'relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['accuracy', 'Recall'])

start = timer()

history = model.fit(train_g, 
                    epochs=5, 
                    validation_data=val_g, 
                    workers = 7
                   )

end = timer()
elapsed = end - start
print('Total Time Elapsed: ', int(elapsed//60), ' minutes ', (round(elapsed%60)), ' seconds')

Epoch 1/5
43/43 [==============================] - 80s 2s/step - loss: 0.5635 - accuracy: 0.7311 - recall: 0.6776 - val_loss: 0.5086 - val_accuracy: 0.7572 - val_recall: 0.7993
Epoch 2/5
43/43 [==============================] - 79s 2s/step - loss: 0.5117 - accuracy: 0.7645 - recall: 0.7474 - val_loss: 0.5049 - val_accuracy: 0.7605 - val_recall: 0.7925
Epoch 3/5
43/43 [==============================] - 79s 2s/step - loss: 0.4971 - accuracy: 0.7793 - recall: 0.7548 - val_loss: 0.5022 - val_accuracy: 0.7622 - val_recall: 0.8061
Epoch 4/5
43/43 [==============================] - 80s 2s/step - loss: 0.4807 - accuracy: 0.7786 - recall: 0.7325 - val_loss: 0.4965 - val_accuracy: 0.7673 - val_recall: 0.7925
Epoch 5/5
43/43 [==============================] - 80s 2s/step - loss: 0.4672 - accuracy: 0.7838 - recall: 0.7489 - val_loss: 0.4944 - val_accuracy: 0.7791 - val_recall: 0.7177
Total Time Elapsed:  6  minutes  48  seconds


In [31]:
model = models.Sequential()
model.add(layers.Conv2D(64, 3, activation = 'relu', input_shape=(256,256,3)))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(256, activation = 'relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['accuracy', 'Recall'])

start = timer()

history = model.fit(train_g, 
                    epochs=5, 
                    validation_data=val_g, 
                    workers = 7
                   )

end = timer()
elapsed = end - start
print('Total Time Elapsed: ', int(elapsed//60), ' minutes ', (round(elapsed%60)), ' seconds')

Epoch 1/5
43/43 [==============================] - 231s 5s/step - loss: 0.5679 - accuracy: 0.7266 - recall: 0.6805 - val_loss: 0.5154 - val_accuracy: 0.7589 - val_recall: 0.7483
Epoch 2/5
43/43 [==============================] - 234s 5s/step - loss: 0.5223 - accuracy: 0.7660 - recall: 0.7355 - val_loss: 0.5715 - val_accuracy: 0.7487 - val_recall: 0.8980
Epoch 3/5
43/43 [==============================] - 231s 5s/step - loss: 0.5076 - accuracy: 0.7637 - recall: 0.7340 - val_loss: 0.5879 - val_accuracy: 0.7470 - val_recall: 0.9048
Epoch 4/5
43/43 [==============================] - 230s 5s/step - loss: 0.4998 - accuracy: 0.7734 - recall: 0.7340 - val_loss: 0.5016 - val_accuracy: 0.7589 - val_recall: 0.7925
Epoch 5/5
43/43 [==============================] - 228s 5s/step - loss: 0.4902 - accuracy: 0.7682 - recall: 0.7325 - val_loss: 0.4967 - val_accuracy: 0.7791 - val_recall: 0.7551
Total Time Elapsed:  19  minutes  45  seconds


In [32]:
model = models.Sequential()
model.add(layers.Conv2D(64, 3, activation = 'relu', input_shape=(256,256,3)))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(256, activation = 'relu'))
model.add(layers.Dense(256, activation = 'relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['accuracy', 'Recall'])

start = timer()

history = model.fit(train_g, 
                    epochs=5, 
                    validation_data=val_g, 
                    workers = 7
                   )

end = timer()
elapsed = end - start
print('Total Time Elapsed: ', int(elapsed//60), ' minutes ', (round(elapsed%60)), ' seconds')

Epoch 1/5
43/43 [==============================] - 224s 5s/step - loss: 0.5736 - accuracy: 0.7147 - recall: 0.6508 - val_loss: 0.5582 - val_accuracy: 0.7454 - val_recall: 0.8946
Epoch 2/5
43/43 [==============================] - 224s 5s/step - loss: 0.5145 - accuracy: 0.7756 - recall: 0.7415 - val_loss: 0.4981 - val_accuracy: 0.7774 - val_recall: 0.7687
Epoch 3/5
43/43 [==============================] - 225s 5s/step - loss: 0.5088 - accuracy: 0.7682 - recall: 0.7251 - val_loss: 0.5755 - val_accuracy: 0.7386 - val_recall: 0.8912
Epoch 4/5
43/43 [==============================] - 228s 5s/step - loss: 0.4976 - accuracy: 0.7689 - recall: 0.7221 - val_loss: 0.5018 - val_accuracy: 0.7757 - val_recall: 0.7619
Epoch 5/5
43/43 [==============================] - 232s 5s/step - loss: 0.4951 - accuracy: 0.7801 - recall: 0.7429 - val_loss: 0.5061 - val_accuracy: 0.7690 - val_recall: 0.8129
Total Time Elapsed:  19  minutes  20  seconds


In [33]:
model = models.Sequential()
model.add(layers.Conv2D(64, 3, activation = 'relu', input_shape=(256,256,3)))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(256, activation = 'relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['accuracy', 'Recall'])

start = timer()

history = model.fit(train_g, 
                    epochs=5, 
                    validation_data=val_g, 
                    workers = 7
                   )

end = timer()
elapsed = end - start
print('Total Time Elapsed: ', int(elapsed//60), ' minutes ', (round(elapsed%60)), ' seconds')

Epoch 1/5
43/43 [==============================] - 240s 6s/step - loss: 0.5649 - accuracy: 0.7266 - recall: 0.6895 - val_loss: 0.5360 - val_accuracy: 0.7690 - val_recall: 0.8810
Epoch 2/5
43/43 [==============================] - 254s 6s/step - loss: 0.5254 - accuracy: 0.7533 - recall: 0.7236 - val_loss: 0.4926 - val_accuracy: 0.7673 - val_recall: 0.7721
Epoch 3/5
43/43 [==============================] - 247s 6s/step - loss: 0.5106 - accuracy: 0.7645 - recall: 0.7281 - val_loss: 0.4943 - val_accuracy: 0.7808 - val_recall: 0.7483
Epoch 4/5
43/43 [==============================] - 245s 6s/step - loss: 0.5046 - accuracy: 0.7734 - recall: 0.7355 - val_loss: 0.4881 - val_accuracy: 0.7740 - val_recall: 0.7687
Epoch 5/5
43/43 [==============================] - 279s 6s/step - loss: 0.5013 - accuracy: 0.7682 - recall: 0.7370 - val_loss: 0.4898 - val_accuracy: 0.7825 - val_recall: 0.7687
Total Time Elapsed:  21  minutes  34  seconds


## 2x2x1x1C1D Adam

In [34]:
model = models.Sequential()
model.add(layers.Conv2D(64, 3, activation = 'relu', input_shape=(256,256,3)))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(256, activation = 'relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.Adam(),
              metrics=['accuracy', 'Recall'])

start = timer()

history = model.fit(train_g, 
                    epochs=5, 
                    validation_data=val_g, 
                    workers = 7
                   )

end = timer()
elapsed = end - start
print('Total Time Elapsed: ', int(elapsed//60), ' minutes ', (round(elapsed%60)), ' seconds')

Epoch 1/5
43/43 [==============================] - 267s 6s/step - loss: 0.6146 - accuracy: 0.6776 - recall: 0.6984 - val_loss: 0.4986 - val_accuracy: 0.7774 - val_recall: 0.7245
Epoch 2/5
43/43 [==============================] - 296s 7s/step - loss: 0.5243 - accuracy: 0.7675 - recall: 0.7103 - val_loss: 0.5341 - val_accuracy: 0.7454 - val_recall: 0.8129
Epoch 3/5
43/43 [==============================] - 307s 7s/step - loss: 0.5114 - accuracy: 0.7675 - recall: 0.7221 - val_loss: 0.5156 - val_accuracy: 0.7639 - val_recall: 0.8401
Epoch 4/5
43/43 [==============================] - 295s 7s/step - loss: 0.5015 - accuracy: 0.7741 - recall: 0.7207 - val_loss: 0.5066 - val_accuracy: 0.7605 - val_recall: 0.7959
Epoch 5/5
43/43 [==============================] - 307s 7s/step - loss: 0.5096 - accuracy: 0.7734 - recall: 0.7489 - val_loss: 0.5438 - val_accuracy: 0.7858 - val_recall: 0.7381
Total Time Elapsed:  25  minutes  6  seconds


## 2x2x1x1C1D Adam / BatchNorm

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(96, 3, activation = 'relu', input_shape=(256,256,3)))
model.add(layers.Conv2D(96, 3, activation = 'relu'))
model.add(BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(32, 3, activation = 'relu'))
model.add(BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(256, activation = 'relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.Adam(),
              metrics=['accuracy'])

start = timer()

history = model.fit(train_g, 
                    epochs=5, 
                    validation_data=val_g, 
                    workers = 7
                   )

end = timer()
elapsed = end - start
print('Total Time Elapsed: ', int(elapsed//60), ' minutes ', (round(elapsed%60)), ' seconds')